In [4]:
import requests
import json
import time
import random
from bs4 import BeautifulSoup as BS

# Headers

In [5]:
useful_user_agent = [
'Mozilla/5.0(Macintosh;U;IntelMacOSX10_6_8;en-us)AppleWebKit/534.50(KHTML,likeGecko)Version/5.1Safari/534.50',

'Mozilla/5.0(Windows;U;WindowsNT6.1;en-us)AppleWebKit/534.50(KHTML,likeGecko)Version/5.1Safari/534.50',

'Mozilla/5.0(compatible;MSIE9.0;WindowsNT6.1;Trident/5.0',

'Mozilla/4.0(compatible;MSIE8.0;WindowsNT6.0;Trident/4.0)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT6.0)',

'Mozilla/4.0(compatible;MSIE6.0;WindowsNT5.1)',

'Mozilla/5.0(Macintosh;IntelMacOSX10.6;rv:2.0.1)Gecko/20100101Firefox/4.0.1',

'Mozilla/5.0(WindowsNT6.1;rv:2.0.1)Gecko/20100101Firefox/4.0.1',

'Opera/9.80(Macintosh;IntelMacOSX10.6.8;U;en)Presto/2.8.131Version/11.11',

'Opera/9.80(WindowsNT6.1;U;en)Presto/2.8.131Version/11.11',

'Mozilla/5.0(Macintosh;IntelMacOSX10_7_0)AppleWebKit/535.11(KHTML,likeGecko)Chrome/17.0.963.56Safari/535.11',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;Maxthon2.0)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;TencentTraveler4.0)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;TheWorld)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;Trident/4.0;SE2.XMetaSr1.0;SE2.XMetaSr1.0;.NETCLR2.0.50727;SE2.XMetaSr1.0)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;360SE)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1;AvantBrowser)',

'Mozilla/4.0(compatible;MSIE7.0;WindowsNT5.1)']

def download_comment_header():
    return {
    'authority': 'api.bilibili.com',
    'method': 'GET',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'referer': 'https://www.bilibili.com/',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': random.choice(useful_user_agent)
}

In [6]:
download_comment_header()

{'authority': 'api.bilibili.com',
 'method': 'GET',
 'scheme': 'https',
 'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'zh-CN,zh;q=0.9',
 'referer': 'https://www.bilibili.com/',
 'sec-fetch-dest': 'script',
 'sec-fetch-mode': 'no-cors',
 'sec-fetch-site': 'same-site',
 'user-agent': 'Mozilla/5.0(Windows;U;WindowsNT6.1;en-us)AppleWebKit/534.50(KHTML,likeGecko)Version/5.1Safari/534.50'}

# 获得所有视频

In [8]:
#upid: up主的个人id， ps: page size, pn: page number
def all_video_url(upid, ps, pn):
    return "https://api.bilibili.com/x/space/arc/search?mid=" + str(upid) + "&ps=" + str(ps) + "&tid=0&pn=" + str(pn) + "&keyword=&order=pubdate&jsonp=jsonp"

In [9]:
#upid: up主的个人id， ps: page size
#return: 以视频avid为key的dict，包含bvid和title
def get_info_all_video(upid, ps):
#     ps = 50
#     upid = 546195
    r = requests.get(all_video_url(upid,ps,1), headers = all_video_header)

    jsonText = json.loads(r.text)
    totalVideoNums = jsonText['data']['page']['count']
    page = (totalVideoNums - 1)//ps + 1
    videoInfo = {}
    vlist = jsonText['data']['list']['vlist']
    print("-------------Doing page{}---------------".format(1))
    for v in vlist:
        videoInfo[v['aid']] = [v['bvid'], v['title']]
    for i in range(2, page + 1):
        r = requests.get(all_video_url(upid,ps,i), headers = all_video_header)
        jsonText = json.loads(r.text)
        vlist = jsonText['data']['list']['vlist']
        print("-------------Doing page{}---------------".format(i))
        for v in vlist:
            videoInfo[v['aid']] = [v['bvid'], v['title']]
    print('Done!')
    return videoInfo

In [10]:
#pn: page number, oid: 视频id(av)
def get_comment_url(pn, oid):
    return 'https://api.bilibili.com/x/v2/reply?&jsonp=jsonp&pn={}&ps=50&type=1&oid={}&sort=2&_=1603036280474'.format(pn, oid)

In [11]:
#oid: 视频id(av)
#return: [(用户名，评论)，(用户名，评论)...]
def get_comment(oid):
    t = time.time()
    pn = 1
#     oid = 457025973
    try:
        r = requests.get(get_comment_url(pn,oid), headers = download_comment_header())
        jsonText = json.loads(r.text)
        count = jsonText['data']['page']['count']
        comments = []
        pages = count // 20 + (0 if count % 20 == 0 else 1)
        for i in jsonText['data']['replies']:
            comments.append(i['content']['message'])
        while(pn < pages):
            #不sleep会被封，封了需要换user-agent
            time.sleep(1)
            pn += 1
            r = requests.get(get_comment_url(pn,oid), headers = download_comment_header)
            jsonText = json.loads(r.text)
            for i in jsonText['data']['replies']:
                comments.append((i['member']['uname'], i['content']['message']))
        len(comments)
        print(len(comments))
    except:
        print("GG")  
    print(time.time() - t)
    return comments

In [16]:
#TODO: json['data']返回的是所有分视频的cid信息，需要额外操作
#oid: 视频id(av)
def get_cid(oid):
    url = 'http://api.bilibili.com/x/player/pagelist?callback=jsonp&aid={}'.format(oid)
    r = requests.get(url = url, headers = download_comment_header())
    return json.loads(r.text)['data'][0]['cid']

In [13]:
#cid: 细分视频id
def get_danmu_url(cid):
    url = 'https://comment.bilibili.com/{}.xml'.format(cid)
    return url

In [32]:
#oid: 视频id(av)
#return: [time, timestamp, userid, danmuid, 弹幕]
def get_danmu(oid):
#     oid = 457025973
    url = get_danmu_url(get_cid(oid))
    r = requests.get(url = url, headers = download_comment_header())
    formated_file = BS(r.content, "lxml")
    danmu = []
    for dm in formated_file.find_all('d'):
        text = dm.get_text()
        info = dm["p"].split(",")
        if text == '':
            continue
        danmu.append([info[0], info[4], info[6], info[7], text])
    
    return danmu

In [44]:
f = open("comments.txt", "w")
t = time.time()
oid = 457025973
comment = get_comment(oid)
print(time.time()-t)
print(len(comment))

2231
132.45176649093628
132.45373058319092
2231


In [60]:
import codecs
def output(danmu, comment):
    f = codecs.open("danmus.txt", "w", "utf-8")
    for i in danmu:
        f.write(i.encode('utf-8').decode('utf-8'))
        f.write("\n")
    f.close()
    f = codecs.open("comments.txt", "w", "utf-8")
    for i in comment:
        f.write(i[0])
        f.write(": ")
        f.write(i[1])
        f.write("\n")
    f.close()

In [17]:
oid = 457025973
url = get_danmu_url(get_cid(oid))
r = requests.get(url = url, headers = download_comment_header())


In [30]:
formated_file = BS(r.content, "lxml")
danmu = []
for dm in formated_file.find_all('d'):
    text = dm.get_text()
    info = dm["p"].split(",")
    if text == '':
        continue
    danmu.append([info[0], info[4], info[6], info[7], text])

In [29]:
type(dm)
dm["p"].split(",")

['95.41200',
 '1',
 '25',
 '16777215',
 '1600514312',
 '0',
 'b6168067',
 '38468131170549765']

In [39]:
import codecs
f = codecs.open("danmus.txt", "r", "utf-8")

In [40]:
danmu = f.read().split()
danmu[3]

'重庆人表示有笑到'

In [46]:
import jieba
from queue import PriorityQueue as PQ
' '.join(jieba.cut(danmu[3], cut_all = True))

'重庆 人 表示 有 笑 到'

In [50]:
dic = {}
for i in danmu:
    fenci = ' '.join(jieba.cut(i)).split()
    for j in fenci:
        if j in dic:
            dic[j] += 1
        else:
            dic[j] = 1
pq = PQ()
index = 0
for i in dic:
    index += 1
    pq.put([10000 - dic[i], index, i])
while not pq.empty():
    a = pq.get()
    print(10000 - a[0], a[2])

389 ？
283 了
257 哈哈哈
235 ！
165 ゴ
155 来
150 的
103 ，
98 哈哈哈哈
94 我
94 。
78 重庆
52 啊
46 是
45 人
45 你
45 《
45 》
40 !
32 狂喜
27 哈哈
27 不
26 吃
25 在
25 吗
24 朋克
24 好
22 这
22 吃面
22 老
21 笑
21 都
21 ~
20 热乎
20 ko
20 这个
20 模拟器
19 番茄
19 no
19 dio
19 da
18 开坑
18 1300
17 有
17 啦
17 （
17 不是
17 逼
16 我们
16 每
16 一家
16 死
16 牛
15 ）
15 游戏
15 就
15 ～
14 店
14 也
13 吧
13 一切
12 …
12 吃遍
12 万
12 什么
12 DNA
12 呢
11 ?
11 哦
11 1300w
10 到
10 脸
10 面
10 锻炼
10 逐渐
10 理解
10 赛博
10 超级
9 快递
9 黑社会
9 不怎么
9 走
9 加缪
9 或
9 飞侠
9 钱
9 针
9 戳
8 表示
8 欢迎
8 车
8 可以
8 那个
8 狗
8 确实
8 doge
8 ：
8 边牧
8 像
8 别
8 乐迪
8 闹钟
8 草
8 危
8 大木
8 没用
8 芜湖
8 小
8 上海
7 说
7 为
7 拜拜
7 光
7 让
7 很
7 君
7 男
7 爱
7 拉面
7 被
7 没
7 合影
7 弹幕
7 阿姨
7 压一压
7 某幻
7 哇
7 把
7 第五
7 元素
7 喜欢
7 好像
7 怎么
6 白
6 水果
6 一个
6 还
6 正道
6 先生
6 一点
6 啊啊啊
6 别念
6 动
6 1
6 叫
6 爷青回
6 买
6 左右
6 Dio
6 给
6 那
6 太棒了
6 飞
6 了解
6 —
5 会
5 老师
5 看
5 就是
5 打
5 呆
5 克劳德
5 所以
5 字幕
5 酒驾
5 害人害己
5 两行
5 泪
5 撞
5 不要
5 城
5 为什么
5 云
5 云吞
5 公司
5 “
5 ”
5 开始
5 咕咕
5 下巴
5 口绿
5 行车
5 规范
5 ga
5 上
5 杀手
5 粉丝
5 盒
5 飞船
5 反应
5 快
5 请
5 可
4 停车
4 撞坏
4 员
4 没有
4 电